# Quick Start Tutorial of Scenario Simulation

Welcome to try out MetaDrive & ScenarioNet!

The simulation supports two running modes:

1. **With 3D rendering functionality**: MetaDrive can easily install and run in personal computer, but may need special treatments for 3D rendering in headless machine and cloud servers.

2. **Without 3D rendering functionality**: MetaDrive can easily install and run in any machine. In this Colab notebook, we demonstrate MetaDrive in this mode and the renderer will be the **2D** **Pygame** renderer.

In this tutorial, we will navigate you through the installation and some basic functionality of the simulator!

## Installation

You can install MetaDrive easily.

In [ ]:
#@title Collect the MetaDrive & ScenarioNet
# NOTE: If you are running this notebook locally with installtion finished, this step is not required.
RunningInCOLAB = 'google.colab' in str(get_ipython()) # Detect if it is running in Colab
if RunningInCOLAB:
    %pip install git+https://github.com/metadriverse/metadrive.git
    %pip install git+https://github.com/metadriverse/scenarionet.git

Next, let's create a 2D visualization tool for recording the scenario in GIF.

In [ ]:
# visualization
from IPython.display import Image as IImage
import pygame
import numpy as np
from PIL import Image

def make_GIF(frames, name="demo.gif"):
    print("Generate gif...")
    imgs = [frame for frame in frames]
    imgs = [Image.fromarray(img) for img in imgs]
    imgs[0].save(name, save_all=True, append_images=imgs[1:], duration=50, loop=0)

## Configuration

Let's import some modules and specify the dataset directory.
**Note: if your machine supports 3D OpenGL rendering, you can turn on the *threeD_render* flag in the following cell. It will render both the 2D results and 3D results.**

In [ ]:
#@title Make some configurations and import some modules
from metadrive.engine.engine_utils import close_engine
close_engine()
from metadrive.pull_asset import pull_asset
pull_asset(False)
# NOTE: usually you don't need the above lines. It is only for avoiding a potential bug when running on colab

from metadrive.engine.asset_loader import AssetLoader
from metadrive.policy.replay_policy import ReplayEgoCarPolicy
from metadrive.envs.scenario_env import ScenarioEnv
import os

threeD_render=False # turn on this to enable 3D render. It only works when you have a screen and not running on Colab.
threeD_render=threeD_render and not RunningInCOLAB
os.environ["SDL_VIDEODRIVER"] = "dummy" # Hide the pygame window
waymo_data =  AssetLoader.file_path(AssetLoader.asset_path, "waymo", unix_style=False) # Use the built-in datasets with simulator
nuscenes_data =  AssetLoader.file_path(AssetLoader.asset_path, "nuscenes", unix_style=False) # Use the built-in datasets with simulator

In [ ]:
os.listdir(waymo_data) # there are 3 waymo scenario file with a 'dataset_summary.pkl'

In [ ]:
os.listdir(nuscenes_data) # there are 10 nuscenes scenario file with a 'dataset_summary.pkl' and a 'dataset_summary.pkl'

## Simulate one Waymo scenario
The simulation interface is in gym-style and let's create a environment first. 
By specifying the *data_directory*, we can load the Waymo dataset to simulation. *num_scenarios* is used to determine how many scenarios are loaded from the datasets. Here we only load one scenario from the Waymo dataset.

In [ ]:
env = ScenarioEnv(
    {
        "manual_control": False,
        "reactive_traffic": False,
        "use_render": threeD_render,
        "agent_policy": ReplayEgoCarPolicy,
        "data_directory": waymo_data,
        "num_scenarios": 1
    }
)

Now the simulation can run with *env.step()* and *env.reset(seed=scenario-index)*. Their functions are as follows.

- The *env.reset(seed=scenario-index)* tells the simulator to remove all existing objects created in last episode, create a new scenario with index *scenario-index* and start a new episode.
**As we only have one scenario loaded to the simulator, the *scenario-index* can only be 1 in this example.**


- *env.step()* will progress the simulation by one step (0.1 second) and return the new observation, reward and termination flag. It takes an action as input which is a 2-dim vector representing the throttle and steering angle for the ego car.
As we are using the *ReplayEgoCarPolicy* here, the ego car will not take the external action accepted from *env.step()*.
**Instead, the ego car will follow the recorded trajectory. Thus the following code is for playing one recorded scenario including maps and trajetcories.**

In [ ]:
# @title Run Simulation

o, _ = env.reset(seed=0)
frames = []
for i in range(1, 100000):
    o, r, tm, tc, info = env.step([1.0, 0.])
    frames.append(env.render(mode="top_down",film_size=(1200, 1200)))
    if tm or tc:
        break
env.close()

make_GIF(frames)
# visualization
IImage(open("demo.gif", 'rb').read())

## Simulate Multiple nuScenes scenarios
For simulating multiple scenarios, just modify the *num_scenarios* and use *env.reset(seed=target_index)* to select the scenario of interest.
This example loading all scenarios into simulator but only simulate and visualize 2 of them.

In [ ]:
env = ScenarioEnv(
    {
        "manual_control": False,
        "reactive_traffic": False,
        "use_render": False,
        "agent_policy": ReplayEgoCarPolicy,
        "data_directory": nuscenes_data, # use nuscenes data
        "num_scenarios": 10, # load 10 scenarios
    }
)

for seed in range(2): # only simulate the first 2 scenarios
    print("\nSimulate Scenario: {}".format(seed))
    o, _ = env.reset(seed=seed)
    frames = []
    for i in range(1, 100000):
        o, r, tm, tc, info = env.step([1.0, 0.])
        frames.append(env.render(mode="top_down",film_size=(4000, 4000), screen_size=(500, 500)))
        if tm or tc:
            make_GIF(frames, name="scenario_{}.gif".format(seed))
            break

env.close()

In [ ]:
%pip install imageio

import imageio
import numpy as np    

#Create reader object for the gif
gif1 = imageio.get_reader('scenario_0.gif')
gif2 = imageio.get_reader('scenario_1.gif')


#If they don't have the same number of frame take the shorter
number_of_frames = min(gif1.get_length(), gif2.get_length())-1

#Create writer object
new_gif = imageio.get_writer('output.gif')

for frame_number in range(number_of_frames):
    img1 = gif1.get_next_data()
    img2 = gif2.get_next_data()
    #here is the magic
    new_image = np.hstack((img1, img2))
    new_gif.append_data(new_image)

gif1.close()
gif2.close()    
new_gif.close()

# visulization
IImage(open("output.gif", 'rb').read())